# **Отработка функций**

## *Подключение модулей*

Не все модули подключились из-за ограниченной функциональности JupyterLite.
Через Anaconda подключаются все модули для аналитики.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(style="darkgrid")

## *Занесение данных в переменную df*

In [ ]:
df = pd.read_csv('fortune500.csv')

Просмотр первых строк **df.head(3)**, где 3 - это количество строк.
Просмотр последних строк **df.tail(3)** 

In [ ]:
df.head(3)

## *Переименование столбцов*

Переименовываем столбы для удобства обращения:

In [ ]:
df.columns = ['year', 'rank', 'company', 'revenue', 'profit']

## *Проверка целостности, типов данных*

In [ ]:
len(df)

In [ ]:
df.dtypes

Столбец **profit** должен быть типа float. Требуется найти все строки, в которых указаны неверные типы данных.
В переменную **non_numberic_profits** заносим строки, в которых значение столбцов **profit** согласно RegEx:

In [ ]:
non_numberic_profits = df.profit.str.contains('[^0-9.-]')
df.loc[non_numberic_profits].head()

In [ ]:
set(df.profit[non_numberic_profits])

In [ ]:
len(df.profit[non_numberic_profits])

После анализа - удаление строк с N.A.:

In [ ]:
df = df.loc[~non_numberic_profits]
df.profit = df.profit.apply(pd.to_numeric)

Строим график:

In [ ]:
group_by_year = df.loc[:, ['year', 'revenue', 'profit']].groupby('year')

avgs = group_by_year.mean()
x = avgs.index
y1 = avgs.profit

def plot(x, y, ax, title, y_label):
    ax.set_title(title)
    ax.set_ylabel(y_label)
    ax.plot(x, y)
    ax.margins(x=0, y=0)

In [ ]:
fig, ax = plt.subplots()
plot(x, y1, ax, 'Increase in mean Fortune 500 company profits from 1955 to 2005', 'Profit (millions)')

График доходов:

In [ ]:
y2 = avgs.revenue
fig, ax = plt.subplots()
plot(x, y2, ax, 'Increase in mean Fortune 500 company revenues from 1955 to 2005', 'Revenue (millions)')

С небольшой помощью Stack Overflow мы можем наложить эти графики с +/- их стандартными отклонениями:

In [ ]:
def plot_with_std(x, y, stds, ax, title, y_label):
    ax.fill_between(x, y - stds, y + stds, alpha=0.2)
    plot(x, y, ax, title, y_label)

fig, (ax1, ax2) = plt.subplots(ncols=2)
title = 'Increase in mean and std Fortune 500 company %s from 1955 to 2005'
stds1 = group_by_year.std().profit.values
stds2 = group_by_year.std().revenue.values

plot_with_std(x, y1.values, stds1, ax1, title % 'profits', 'Profit (millions)')
plot_with_std(x, y2.values, stds2, ax2, title % 'revenues', 'Revenue (millions)')

fig.set_size_inches(14, 4)
fig.tight_layout()